<h1>客户分类<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#主要结论前置" data-toc-modified-id="主要结论前置-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>主要结论前置</a></span></li><li><span><a href="#客户分类目标" data-toc-modified-id="客户分类目标-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>客户分类目标</a></span></li><li><span><a href="#RFM-模型" data-toc-modified-id="RFM-模型-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>RFM 模型</a></span></li><li><span><a href="#KMeans-聚类算法" data-toc-modified-id="KMeans-聚类算法-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>KMeans 聚类算法</a></span></li></ul></div>

*Author: json.wong.work@gmail.com*

*Index: https://github.com/jsonww/cosmetics-shop-exploratory-analysis*

#### 主要结论前置

1. 按传统 RFM 模型分类，重要客户与一般客户数量3/7开，销售额4/6开。客户质量两极分化：一般发展客户和一般挽留客户占比高达 62%，应该重视这部分用户的促活；重要价值客户占比 12% 仅次于前两者，应重视这部分用户的服务，注意留存和LTV的提升。
2. 最近购买时间是客户群体间的主要差异，应针对不同程度久未购买客户采取不同程度的唤醒/召回措施。

#### 客户分类目标

本次客户分类的目标是对该季度内有过订单行为的客户进行价值细分，以期实现不同类型价值客户的精准运营，达到降本增效。
- 增效：针对优质客群的提供高质量服务，充分发掘潜在优质客群使其向优质客群转化。
- 降本：控制低质量客群的运营、投放成本。

本次客户分类使用了**基于业务的 RFM 模型**和**基于机器学习的 KMeans 聚类算法**两种方式，以期对照结合、充分挖掘数据中的有效信息。

#### RFM 模型

构建 RFM 模型，对客户（本季度有过消费行为的用户）进行分类，方便跟踪各层用户的特征及变化情况，以针对性运营提高效率。

目标指标：
- Recency: 最近一次消费时间间隔
- Frequency：消费频率
- Monetary：消费金额

划分方式：以均值作为划分标准，将 RFM 三个指标数据转化为高、低两个等级变量，根据用户每项指标等级，划分为 8 类具不同消费特征的群体，由此直观反映各类客户的价值和重要性，并可根据相应特征采取针对性策略，可操作性强。

|R|F|M|划分群体类型|特点|
|-|-|-|-|-|
|高|高|高|重要价值客户|高端客户，各项指标都优于平均，质量最优|
|低|高|高|重要保持客户|最近购买日期早于平均最近购买日期，存在流失的可能性|
|高|低|高|重要发展客户|消费次数低于均值，用户粘性不高|
|低|低|高|重要挽留客户|消费次数低于均值和最近购买日期早于平均最近购买日期，流失风险大|
|高|高|低|一般价值客户|近期消费、频率高于平均，但消费金额低于平均|
|低|高|低|一般保持客户|频率高于平均、消费金额低于平均，近期无消费，存在流失的可能性|
|高|低|低|一般发展客户|近期消费，但频率和消费金额均低于平均，可能是试水新用户或粘性不高的老用户|
|低|低|低|一般挽留客户|各项指标都低于平均|

注：由于消费金额 M 是衡量用户价值更重要的属性，根据消费高金额 M 的高低又划分为重要客户和一般客户。

In [1]:
'''
输出与处理与特征工程
'''
import numpy as np
import pandas as pd

from pyecharts.charts import *
from pyecharts.components import Table
from pyecharts.commons.utils import JsCode
import pyecharts.options as opts
from pyecharts.globals import ThemeType

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import metrics


def draw_pareto(data, tittle='帕累托图', x_name='',
                colors=['#D53A35', '#2F4554', '#61A0A8', '#D48265', '#749F83', '#CA8622', '#BDA29A', '#6E7074'], is_show=True):
    '''
    Draw pareto.
    '''
    data[f'{data.columns[0]}-累计占比'] = np.round(data.values[:, 0].cumsum() / data.values[:, 0].sum(), 2)
    bar_data_pair = []
    bar_x_axis = data.index.astype(str).tolist()
    bar_y_axis = data.iloc[:, 0].tolist()
    for k, v, c in zip(bar_x_axis, bar_y_axis, colors):
        bar_data_pair.append(
            opts.BarItem(
                name=k,
                value=v,
                itemstyle_opts=opts.ItemStyleOpts(color=c)
            )
        )

    bar = (
        Bar()
        .set_global_opts(
            title_opts=opts.TitleOpts(title=tittle),
            tooltip_opts=opts.TooltipOpts(
                is_show=True,
                trigger="axis",
                axis_pointer_type="cross",
            ),
            xaxis_opts=opts.AxisOpts(
                name=x_name,
                name_location='center',
                name_gap=30,
                type_="category",
                axispointer_opts=opts.AxisPointerOpts(is_show=True, type_="shadow"),
            ),
            yaxis_opts=opts.AxisOpts(
                name=data.columns[0],
                type_="value",
                axislabel_opts=opts.LabelOpts(formatter="{value}"),
                axistick_opts=opts.AxisTickOpts(is_show=True),
            ),
            legend_opts=opts.LegendOpts(
                type_="scroll",
            ),
        )
        .extend_axis(
            yaxis=opts.AxisOpts(
                name=data.columns[1],
                type_="value",
                min_=0,
                max_=1.1,
                axislabel_opts=opts.LabelOpts(formatter="{value}"),
            )
        )
        .add_xaxis(bar_x_axis)
        .add_yaxis(
            data.columns[0], 
            bar_data_pair,
            z=0,
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )

    cumsum_line = (
        Line()
        .add_xaxis(data.index.astype(str).tolist())
        .add_yaxis(
            data.columns[1], 
            data.iloc[:, 1].tolist(),
            yaxis_index=1,
            linestyle_opts=opts.LineStyleOpts(width=1),
            markline_opts=opts.MarkLineOpts(
                data=[opts.MarkLineItem(y=0.8)],
                linestyle_opts=opts.LineStyleOpts(
                    width=1,
                    type_='dashed',
                    opacity=0.8,
                ),
                symbol='none',
                label_opts=opts.LabelOpts(is_show=False),
            ),
            z=1
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )
    
    return bar.overlap(cumsum_line).render_notebook() if is_show else bar.overlap(cumsum_line)


# 读取数据
DATA_PATH = '../data/'
CLEAN_PATH = '../clean/'
CACHE_PATH = '../cache/'
DATA_FILES = ['2019-Oct.csv', '2019-Nov.csv', '2019-Dec.csv',]
cleaned_target = f"{CLEAN_PATH}processed_data.h5"

data = pd.read_hdf(cleaned_target, key='q4')
data = pd.concat([data, pd.get_dummies(data['event_type'])], axis=1)
data['turnovers'] = data['price'] * data['purchase']
data['date'] = data['event_time'].dt.date
data = data.drop(columns=['event_time'])
# data

# 统计消费用户 rfm 指标
user_rfm = pd.DataFrame(data['user_id'].unique(), columns=['user_id'])
user_recently = pd.DataFrame(data[data['purchase'] == 1].groupby('user_id', as_index=False)['date'].apply(lambda x: x.max()))
first_date = data['date'].min()
user_recently['date'] = (user_recently['date'] - first_date).dt.days
user_rfm = pd.merge(left=user_rfm, right=user_recently, how='left', on=['user_id'])
user_frequency = data[['user_id', 'purchase']].groupby('user_id').sum()
user_rfm = pd.merge(left=user_rfm, right=user_frequency, how='left', on=['user_id'])
user_monetary = data[['user_id', 'turnovers']].groupby('user_id').sum()
user_rfm = pd.merge(left=user_rfm, right=user_monetary, how='left', on=['user_id'])
user_rfm.fillna(0, inplace=True)
user_rfm.set_index('user_id', inplace=True)
user_rfm.columns = ['Recently','Frequency', 'Monetary']
# user_rfm


'''
RFM
'''
consumer_rfm = user_rfm[user_rfm['Frequency'] > 0].copy()

# 划分 rfm 等级变量
rfm_divide = {col: consumer_rfm[col].mean() for col in consumer_rfm.columns}  # 取 rfm 中位数
consumer_rfm = pd.DataFrame(consumer_rfm)

for col in consumer_rfm:
    '''等级变量'''
    consumer_rfm[f"_{col}"] = consumer_rfm[col].apply(lambda x: 1 if x > rfm_divide[col] else 0)

# 根据 rfm 等级变量打标签
consumer_rfm['code'] = 100 * consumer_rfm['_Recently'] + 10 * consumer_rfm['_Frequency'] + consumer_rfm['_Monetary']  # 编码
rfm_code2label = {
    111: '重要价值客户',
    11: '重要保持客户',
    101: '重要发展客户',
    1: '重要挽留客户',
    110: '一般价值客户',
    10: '一般保持客户',
    100: '一般发展客户',
    0: '一般挽留客户',
}
consumer_rfm['label'] = consumer_rfm['code'].apply(lambda x: rfm_code2label[x])  # 编码转标签

# 3. 分类效果检验
# 3.1 各类用户占比
rfm_labels_order = ['重要价值客户', '重要保持客户', '重要发展客户', '重要挽留客户', '一般价值客户', '一般保持客户', '一般发展客户', '一般挽留客户']
rfm_labels_vc = consumer_rfm['label'].value_counts().reindex(rfm_labels_order)

rfm_pie = (
    Pie()
    .add(
        '',
        [list(z) for z in zip(rfm_labels_vc.index.tolist(), rfm_labels_vc.values.tolist())],
        radius=[80, 180],
        rosetype="radius", #玫瑰饼图
    )
    .set_global_opts(
        title_opts=opts.TitleOpts('RFM 分类效果'),
        legend_opts=opts.LegendOpts(is_show=False)
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter='{b}: {d}%'))
)

# 3.2 各类用户 rfm 雷达图
label_mean = consumer_rfm.groupby('label')[['Recently', 'Frequency', 'Monetary']].mean()
label_mean = label_mean.reindex(rfm_labels_order)
label_colors = ['#D53A35', '#2F4554', '#61A0A8', '#D48265', '#749F83', '#CA8622', '#BDA29A', '#6E7074']
# 数据做归一化处理
label_mean_mms = MinMaxScaler().fit_transform(label_mean)

label_mean_radar = (
    Radar()
    .add_schema(
        schema=[opts.RadarIndicatorItem(name=col) for col in label_mean.columns]
    )
    .set_global_opts(
        title_opts=opts.TitleOpts("各类客户 RFM 均值雷达图"),
        legend_opts=opts.LegendOpts(pos_bottom='15%'),
    )
)

for i, v in enumerate(label_mean.index):
    label_mean_radar.add(
        v, 
        [label_mean.loc[v].tolist()], 
        color=label_colors[i],
        linestyle_opts=opts.LineStyleOpts(width=2),
    )
    label_mean_radar.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    
label_monetary = pd.DataFrame(consumer_rfm.groupby('label')['Monetary'].sum()).reindex(rfm_labels_order)
label_monetary.columns = ['消费金额']
pareto = draw_pareto(label_monetary, tittle='销售额帕累托图', x_name='客户类型', colors=label_colors, is_show=False)
    
rfm_page = (
    Page()
    .add(rfm_pie, label_mean_radar, pareto)
)
rfm_page.render_notebook()

现象：
- 重要客户占比约 30%，贡献整体销售额的 66%。其中，重要价值客户在整体客户中占比 13.94%，是第三大客户群体，贡献整体销售额的 33%。
- 一般客户占比约 70%，贡献整体销售额的 34%。其中，一般发展客户（32.51%）和一般挽留客户（29.51%）在整体客户中占比明显高于其它群体，是最大的两个客户群体。

**结论：按传统 RFM 模型分类，重要客户与一般客户数量3/7开，销售额4/6开。客户质量两极分化：一般发展客户和一般挽留客户占比高达 62%，应该重视这部分用户的促活；重要价值客户占比 12% 仅次于前两者，应重视这部分用户的服务，注意留存和LTV的提升。**

建议：
- 经营好重要客户，保持重要客户的粘性。例如：
    - '重要价值客户'： VIP 优待 -- 提供更优质的售时售后服务，或进行不定期福利回馈。
    - '重要保持客户'：及时唤醒 -- 采取措施重新唤醒，促进客户复购。若存有联系方式，而可通过固定周期未购买进行消息通知。
    - '重要发展客户' 和 '重要挽留客户'：提高客户粘性 -- 增加对这类客户的曝光量和吸引力。前者可以及时推送优惠活动，后者可以通过调研客户消费需求和满意度调查，对能够改进的地方进行完善。
- 充分开发一般客户，使其向更高层客户群体次转化。可以针对这部分客户的进行消费需求调研和满意度调查，对能够改进的地方进行完善。

#### KMeans 聚类算法

传统 RFM 模型使用指定划分阈值的方式进行分类且明确了群体标签，所以阈值的选取对分类的效果影响很大，客户的类型也局限在明确的群体标签内。为了降低主观影响实现自动分类，依旧选用 RFM 三个指标，使用 KMeans 聚类基于 Calinski-Harabasz Index 自动选择最佳聚类数进行客户分类。

注：Calinski-Harabasz Index 计算耗时明显低于 Silhouette Coefficient，故使用前者进行评估最佳聚类数，再用后者进行最终聚类效果评估。

In [2]:
'''
KMeans
'''
consumer_kmeans = user_rfm[user_rfm['Frequency'] > 0].copy()
consumer_ss = MinMaxScaler().fit_transform(consumer_kmeans)

# 1. 算法自动选择最佳聚类数 K：使用 Calinski-Harabasz Index 指标
best_k, best_calinski = 0, 0
for k in range(2, 9):
    y_pred = KMeans(n_clusters=k, random_state=1234).fit_predict(consumer_ss)
    calinski = metrics.calinski_harabasz_score(consumer_ss, y_pred)
#     print(f"{k}: {calinski}.")
    if calinski > best_calinski:
        best_k, best_calinski = k, calinski

# 2. 聚类：使用算法自动选择的最佳聚类数 K进行聚类
best_model = KMeans(n_clusters=best_k, random_state=1234)
best_model.fit(consumer_ss)
consumer_kmeans['label'] = best_model.labels_

# 3. 聚类效果评估：使用 Silhouette Coefficient 指标
# silhouette = metrics.silhouette_score(consumer_ss, consumer_kmeans['label'])
# print(f"The beset k={best_k}, silhouette score: {silhouette}.")
# # executed in 3m 3s, ……
# # The beset k=6, silhouette score: 0.474299587591174.
# ## 可见聚类效果较好 

km_labels_mean = consumer_kmeans.groupby('label').mean().sort_values(by=['Monetary', 'Frequency', 'Recently'], ascending=False)
labels_mapper = dict(zip(km_labels_mean.index, [chr(ord('A') + i) for i in range(km_labels_mean.shape[0])]))
consumer_kmeans['label'] = consumer_kmeans['label'].map(labels_mapper)
km_labels_mean.index = [chr(ord('A') + i) for i in range(km_labels_mean.shape[0])]
km_labels_vc = consumer_kmeans['label'].value_counts().sort_index()
label_colors = ['#D53A35', '#2F4554', '#61A0A8', '#D48265', '#749F83', '#CA8622', '#CA8622', '#6E7074']

# 3.1 各类型客户占比
kmeans_pie = (
    Pie()
    .add(
        '',
        [list(z) for z in zip(km_labels_vc.index.tolist(), km_labels_vc.values.tolist())],
        radius=[100, 180]
    )
    .set_global_opts(
        title_opts=opts.TitleOpts('K-Means 聚类效果'),
        legend_opts=opts.LegendOpts(is_show=False)
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter='{b}: {d}%'))
)


# 3.2 各类型 RFM 均值
km_labels_mean_mms = MinMaxScaler().fit_transform(km_labels_mean)
km_labels_mean_radar = (
    Radar()
    .add_schema(
        schema=[opts.RadarIndicatorItem(name=col) for col in km_labels_mean.columns]
    )
    .set_global_opts(
        title_opts=opts.TitleOpts("各类用户RFM均值雷达图"),
        legend_opts=opts.LegendOpts(
        pos_right='0%',
            pos_top='middle',
            orient='vertical',
        ),
    )
)

for i, v in enumerate(km_labels_mean.index):
    km_labels_mean_radar.add(
        v, 
        [km_labels_mean.loc[v].tolist()], 
        color=label_colors[i],
        linestyle_opts=opts.LineStyleOpts(width=2),
    )
    km_labels_mean_radar.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    

# 3.3 客户分类簇心
df = pd.DataFrame(np.round(best_model.cluster_centers_, decimals=4))
df.columns = ['R', 'F', 'M']
df = df.sort_values(by=['M', 'F', 'R'], ascending=False)
df.index = [chr(ord('A') + i) for i in range(len(df))]
cluster_center_table = Table(page_title='客户聚类簇心')
cct_headers = ['客户类型 \ 簇心', 'R', 'F', 'M']
cct_rows = [[idx] + val.tolist() for idx, val in df.iterrows()]
cluster_center_table.add(cct_headers, cct_rows)
cluster_center_table.render_notebook()
    
# 3.4 销售额帕累托图    
label_monetary = pd.DataFrame(consumer_kmeans.groupby('label')['Monetary'].sum())
label_monetary.columns = ['贡献销售额']
pareto = draw_pareto(label_monetary, tittle='销售额帕累托图', x_name='客户类型', colors=label_colors, is_show=False)

kmeans_page = (
    Page()
    .add(kmeans_pie, km_labels_mean_radar, pareto)
)
kmeans_page.render_notebook()

In [3]:
df = pd.DataFrame(np.round(best_model.cluster_centers_, decimals=4))
df.columns = ['R', 'F', 'M']
df = df.sort_values(by=['M', 'F', 'R'], ascending=False)
df.index = [chr(ord('A') + i) for i in range(len(df))]
cluster_center_table = Table(page_title='客户聚类簇心')
cct_headers = ['客户类型 \ 簇心', 'R', 'F', 'M']
cct_rows = [[idx] + val.tolist() for idx, val in df.iterrows()]
cluster_center_table.add(cct_headers, cct_rows)
cluster_center_table.render_notebook()

客户类型 \ 簇心,R,F,M
A,0.9037,0.0257,0.023
B,0.7594,0.0255,0.0215
C,0.6045,0.0248,0.0203
D,0.4333,0.0196,0.0177
E,0.2583,0.0178,0.0162
F,0.0829,0.0157,0.0147


现象：客户被分为 6 个群体，按各群体的 “最近购买时间(R)”、“购买频次(F)”、“购买金额(M)” 指标均值递减依次为 A、B、C、D、E、F、F。其中，根据群体雷达图和各聚类客户簇心可以发现，不同分类的“最近购买时间(R)”的差异明显大于“购买金额(M)”和“购买频次(F)”且层次明显。

**结论：最近购买时间是客户群体间的主要差异，应针对不同程度久未购买客户采取不同程度的唤醒/召回措施。**